# TensorBoardでの画像データの表示

https://www.tensorflow.org/tensorboard/image_summaries?hl=ja

## 概要

TensorFlowイメージ概要APIを使用して、簡単にテンソルと任意の画像を記録し、TensorBoardでそれらを表示することができます。これは、サンプルに非常に役立つことや、あなたの入力データを調べ、または層の重み視覚化および生成されたテンソルをすることができます。モデル開発の過程で役立つ画像として診断データをログに記録することもできます。

このチュートリアルでは、Image SummaryAPIを使用してテンソルを画像として視覚化する方法を学習します。また、任意の画像を取得してテンソルに変換し、TensorBoardで視覚化する方法についても学習します。モデルのパフォーマンスを理解するのに役立つ画像の概要を使用する、シンプルでありながら実際の例を実行します。

## 設定

In [13]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension.
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [14]:
from datetime import datetime
import io
import itertools
from packaging import version

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.7.0


## Fashion-MNISTデータセットをダウンロードする
あなたには分類画像に簡単なニューラルネットワークを構築しようとしているファッション・MNISTのデータセット。このデータセットは、10のカテゴリのファッション製品の70,000の28x28グレースケール画像で構成され、カテゴリごとに7,000の画像があります。

まず、データをダウンロードします。

In [15]:
# Download the data. The data is already divided into train and test.
# The labels are integers representing classes.
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = \
    fashion_mnist.load_data()

# Names of the integer classes, i.e., 0 -> T-short/top, 1 -> Trouser, etc.
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [16]:
print('train_images.shape = ', train_images.shape)
print('train_labels.shape = ', train_labels.shape)
print('test_images.shape = ', test_images.shape)
print('test_labels.shape = ', test_labels.shape)

train_images.shape =  (60000, 28, 28)
train_labels.shape =  (60000,)
test_images.shape =  (10000, 28, 28)
test_labels.shape =  (10000,)


## 単一の画像を視覚化する
Image Summary APIがどのように機能するかを理解するために、TensorBoardのトレーニングセットの最初のトレーニング画像をログに記録するだけです。

その前に、トレーニングデータの形状を調べてください。

In [17]:
print("Shape: ", train_images[0].shape)
print("Label: ", train_labels[0], "->", class_names[train_labels[0]])

Shape:  (28, 28)
Label:  9 -> Ankle boot


データセット内の各画像の形状は、高さと幅を表す形状のランク2テンソル（28、28）であることに注意してください。

しかし、 tf.summary.image()を含む階数4のテンソル期待(batch_size, height, width, channels) 。したがって、テンソルの形状を変更する必要があります。

だから、1枚の画像だけ、だログbatch_size画像がグレースケールのある1ですので、設定channels 1に。

In [18]:
# Reshape the image for the Summary API.
img = np.reshape(train_images[0], (-1, 28, 28, 1))

これで、この画像をログに記録してTensorBoardで表示する準備が整いました。

In [23]:
# Clear out any prior log data.
!rm -rf logs

# Sets up a timestamped log directory.
logdir = "logs/train_data/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Using the file writer, log the reshaped image.
with file_writer.as_default():
  tf.summary.image("Training data", img, step=0)

次に、TensorBoardを使用して画像を調べます。 UIが起動するまで数秒待ちます。

In [24]:
#%tensorboard --logdir logs/train_data

ターミナルから以下を実行し、tensorboard 起動

cd /Users/kaoru/opt/anaconda3/envs/py37_Kaggle_GreatBarrierReef/lib/python3.7/site-packages/tensorboard

python main.py --logdir=/Users/kaoru/Documents/Python_Tensorflow2.0_Guidance/logs/train_data

tensorboard 起動に成功したら、下記にアクセス  
http://localhost:6006/

## 複数の画像を視覚化する
1つのテンソルをログに記録するのは素晴らしいことですが、複数のトレーニング例をログに記録したい場合はどうでしょうか。

単にあなたがにデータを渡すときにログに記録したい画像の数を指定tf.summary.image()

In [25]:
with file_writer.as_default():
  # Don't forget to reshape.
  images = np.reshape(train_images[0:25], (-1, 28, 28, 1))
  tf.summary.image("25 training data examples", images, max_outputs=25, step=0)

# %tensorboard --logdir logs/train_data

## 任意の画像データのログ

以下のコードでは、matplotlibの使用の素敵なグリッドとして最初の25枚の画像を記録しますsubplot()関数を。次に、TensorBoardでグリッドを表示します。

In [26]:
# Clear out prior logging data.
!rm -rf logs/plots

logdir = "logs/plots/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)

def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

def image_grid():
  """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
  # Create a figure to contain the plot.
  figure = plt.figure(figsize=(10,10))
  for i in range(25):
    # Start next subplot.
    plt.subplot(5, 5, i + 1, title=class_names[train_labels[i]])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)

  return figure

# Prepare the plot
figure = image_grid()
# Convert to image and log
with file_writer.as_default():
  tf.summary.image("Training data", plot_to_image(figure), step=0)

#%tensorboard --logdir logs/plots

## 画像分類器の構築
これを実際の例と一緒にまとめます。結局のところ、あなたは機械学習を行うためにここにいて、きれいな写真をプロットするのではありません！

画像の要約を使用して、Fashion-MNISTデータセットの単純な分類器をトレーニングしながらモデルがどの程度うまく機能しているかを理解します。

まず、非常に単純なモデルを作成してコンパイルし、オプティマイザーと損失関数を設定します。コンパイルステップでは、途中で分類子の精度をログに記録することも指定します。

In [27]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

分類器を訓練するとき、それは見るために便利です混同行列を。混同行列は、分類器がテストデータに対してどのように実行されているかについての詳細な知識を提供します。

混同行列を計算する関数を定義します。あなたは便利な使いますScikit-学ぶこれを行うための機能を、その後、matplotlibのを使用して、それをプロットします。

In [28]:
def plot_confusion_matrix(cm, class_names):
  """
  Returns a matplotlib figure containing the plotted confusion matrix.

  Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
  """
  figure = plt.figure(figsize=(8, 8))
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title("Confusion matrix")
  plt.colorbar()
  tick_marks = np.arange(len(class_names))
  plt.xticks(tick_marks, class_names, rotation=45)
  plt.yticks(tick_marks, class_names)

  # Compute the labels from the normalized confusion matrix.
  labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

  # Use white text if squares are dark; otherwise black.
  threshold = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    color = "white" if cm[i, j] > threshold else "black"
    plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  return figure

これで、分類器をトレーニングし、途中で混同行列を定期的に記録する準備が整いました。

これがあなたがすることです：

1. 作成Keras TensorBoardコールバックを基本的な指標をログに記録します
2. 作成Keras LambdaCallbackのすべてのエポックの終わりに混同行列をログに記録します
3. Model.fit（）を使用してモデルをトレーニングし、両方のコールバックを必ず渡すようにします

トレーニングが進むにつれて、下にスクロールしてTensorBoardが起動することを確認します。




In [29]:
# Clear out prior logging data.
!rm -rf logs/image

logdir = "logs/image/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Define the basic TensorBoard callback.
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [30]:
def log_confusion_matrix(epoch, logs):
  # Use the model to predict the values from the validation dataset.
  test_pred_raw = model.predict(test_images)
  test_pred = np.argmax(test_pred_raw, axis=1)

  # Calculate the confusion matrix.
  cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
  # Log the confusion matrix as an image summary.
  figure = plot_confusion_matrix(cm, class_names=class_names)
  cm_image = plot_to_image(figure)

  # Log the confusion matrix as an image summary.
  with file_writer_cm.as_default():
    tf.summary.image("Confusion Matrix", cm_image, step=epoch)

# Define the per-epoch callback.
cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [31]:
# Start TensorBoard.
#%tensorboard --logdir logs/image

# Train the classifier.
model.fit(
    train_images,
    train_labels,
    epochs=5,
    verbose=0, # Suppress chatty output
    callbacks=[tensorboard_callback, cm_callback],
    validation_data=(test_images, test_labels),
)

トレインセットと検証セットの両方で精度が向上していることに注意してください。それは良い兆候です。しかし、モデルはデータの特定のサブセットでどのように機能していますか？

「画像」タブを選択して、ログに記録された混同行列を視覚化します。フルサイズの混同行列を表示するには、左上の[実際の画像サイズを表示する]をオンにします。

デフォルトでは、ダッシュボードには、最後にログに記録されたステップまたはエポックの画像の概要が表示されます。スライダーを使用して、以前の混同行列を表示します。トレーニングが進むにつれて行列がどのように大きく変化するかに注目してください。暗い正方形が対角線に沿って合体し、残りの行列は0と白に向かう傾向があります。これは、トレーニングが進むにつれて分類器が向上していることを意味します。すごい仕事！

混同行列は、この単純なモデルにいくつかの問題があることを示しています。大きな進歩にもかかわらず、シャツ、Tシャツ、プルオーバーは互いに混乱しています。モデルにはさらに作業が必要です。

あなたが興味を持っている場合、でこのモデルを改善しようと、畳み込みネットワーク（CNN）。